In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from tensorflow.keras.datasets import mnist

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
x_train.shape

(60000, 28, 28)

### Channel

CNN 要注意一張圖有多少個 channel, 開始我們因為只有灰階, 所以只有一個 channel。因此我們要轉一下我們的資料格式:

    (28,28) --> (28, 28, 1)

In [7]:
x_train = x_train.reshape(60000, 28, 28, 1) / 255

In [8]:
x_test = x_test.reshape(10000, 28, 28, 1) / 255

In [10]:
x_train[87].shape

(28, 28, 1)

In [11]:
y_train[87]

9

In [12]:
from tensorflow.keras.utils import to_categorical

In [13]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

### 讀入必要的函式

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD

### Step 1 打造函數學習機 (CNN)

In [16]:
model = Sequential()

In [17]:
model.add(Conv2D(16, (3,3), padding='same',
                input_shape=(28,28,1),
                activation='relu'))

In [18]:
# 輸出 16 個 28x28 矩陣
# 事實上是 (28, 28, 16)

In [19]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [20]:
# (14,14,16)

In [21]:
model.add(Conv2D(32, (3,3), padding='same',
                activation='relu'))

In [22]:
# output (14, 14, 32)

In [23]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [24]:
# output (7, 7, 32)

In [25]:
model.add(Conv2D(64, (3,3), padding='same',
                activation='relu'))

In [26]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [27]:
model.add(Flatten())

In [28]:
model.add(Dense(54, activation='relu'))

In [29]:
model.add(Dense(10, activation='softmax'))

### 看一下我們的神經網路

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 576)               0

In [31]:
# 3*3 (權重) + 1 (bias)

(3*3+1)*16

160

In [32]:
#

(3*3*16+1)*32

4640

In [33]:
model.compile(loss='mse', optimizer=SGD(lr=0.087),
             metrics=['accuracy'])

### Step 2. fit

In [34]:
model.fit(x_train, y_train, batch_size=128, epochs=12)

Train on 60000 samples
Epoch 1/12
60000/60000 [==============================] - 7s 125us/sample - loss: 0.0900 - accuracy: 0.1069
Epoch 2/12
60000/60000 [==============================] - 7s 119us/sample - loss: 0.0896 - accuracy: 0.2931
Epoch 3/12
60000/60000 [==============================] - 7s 119us/sample - loss: 0.0892 - accuracy: 0.3473
Epoch 4/12
60000/60000 [==============================] - 7s 119us/sample - loss: 0.0882 - accuracy: 0.4004
Epoch 5/12
60000/60000 [==============================] - 7s 119us/sample - loss: 0.0842 - accuracy: 0.5123
Epoch 6/12
60000/60000 [==============================] - 7s 119us/sample - loss: 0.0472 - accuracy: 0.7167
Epoch 7/12
60000/60000 [==============================] - 7s 119us/sample - loss: 0.0213 - accuracy: 0.8663
Epoch 8/12
60000/60000 [==============================] - 7s 119us/sample - loss: 0.0138 - accuracy: 0.9133
Epoch 9/12
60000/60000 [==============================] - 7s 119us/sample - loss: 0.0109 - accuracy: 0.9313
Epoch

### Step 3. 預測

In [35]:
result = model.predict_classes(x_test)

In [36]:
def my_predict(n):
    print('我可愛的 CNN 預測是', result[n])
    X = x_test[n].reshape(28,28)
    plt.imshow(X, cmap='Greys')

In [39]:
from ipywidgets import interact_manual

In [40]:
interact_manual(my_predict, n=(0, 9999))

interactive(children=(IntSlider(value=4999, description='n', max=9999), Button(description='Run Interact', sty…

<function __main__.my_predict(n)>

In [41]:
score = model.evaluate(x_test, y_test)

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [42]:
loss, acc = score

In [44]:
print('測試資料的正確率為', acc)

測試資料的正確率為 0.961


### 把我們的 model 存起來

In [45]:
model.save('myCNNmodel.h5')